In [1]:
import os
os.chdir('/Users/arshiayousefi/Desktop/ML/armanemo')
os.getcwd()

'/Users/arshiayousefi/Desktop/ML/armanemo'

***Resources***
1.   [Fine-tuning RoBERTa for Topic Classification with Hugging Face Transformers and Datasets Library](https://medium.com/@achillesmoraites/fine-tuning-roberta-for-topic-classification-with-hugging-face-transformers-and-datasets-library-c6f8432d0820)



**Models**
1.   Xlm-Roberta
2.   Roberta
3.   MT5



In [ ]:
!pip install evaluate
!pip install -U accelerate
!pip install -U transformers
!pip install datasets

In [2]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

from datasets import load_dataset, Dataset

import matplotlib

# DataSet

In [4]:
from emotion_detection.preprocess import preprocess_ArmanEmo
# dfff = pd.read_csv('/Users/arshiayousefi/Desktop/ML/armanemo/data/train.tsv', sep='\t', names=['text', 'label'])
# dff = preprocess_ArmanEmo(dfff)

In [5]:
df = pd.read_csv('/Users/arshiayousefi/Desktop/ML/armanemo/data/train.tsv', sep='\t', names=['text', 'label'])
df = preprocess_ArmanEmo(df)
# df.dropna(inplace=True)
# df.to_csv('/Users/arshiayousefi/Desktop/ML/armanemo/data/train_process.csv', index=False)

In [6]:
df.reset_index(drop=True,inplace=True)

In [9]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [10]:
df.to_csv('/Users/arshiayousefi/Desktop/ML/armanemo/data/train_process.csv', index=False)

In [28]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [30]:
pd.read_csv('/Users/arshiayousefi/Desktop/ML/armanemo/data/train_process.csv').isnull().sum()

text     4
label    0
dtype: int64

In [13]:
df.isnull().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [4]:
dataset = load_dataset("csv", data_files='/Users/arshiayousefi/Desktop/ML/armanemo/data/train_process.csv', split="train")

# Convert huggingface dataloader to pandas
df = dataset.to_pandas()
# dataset.set_format("pandas")
# df = dataset["train"][:]
# Dataset.from_pandas(df)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
df.isnull().sum()

text     4
label    0
dtype: int64

In [5]:
from datasets import ClassLabel, DatasetDict

# Example unique labels
unique_labels = list(sorted(df['label'].unique()))

# Create a ClassLabel feature
class_label_feature = ClassLabel(names=unique_labels)

def label_str_to_int(example):
    example['label'] = class_label_feature.str2int(example['label'])
    return example

dataset = dataset.map(label_str_to_int)
dataset = dataset.cast_column('label', class_label_feature)

Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6125 [00:00<?, ? examples/s]

## Split dataset to train, validation and test

In [6]:
dataset_clean = dataset.train_test_split(train_size=0.9, seed=42)
dataset_clean["validation"] = dataset_clean.pop("test")
dataset_clean["test"] = dataset_clean["train"].train_test_split(train_size=0.9, seed=42)["test"]


# Training, Validation and testing datasets
dataset_tmp = dataset_clean.copy()
train_dataset = dataset_tmp['train']
test_dataset = dataset_tmp['test']
val_dataset = dataset_tmp['validation']

# Models

### Configoration

In [8]:
xlm_roberta_id = "xlm-roberta-base"
roberta_id = "roberta-base"
mt5_id = "google/mt5-small"

label_list = list(sorted(df['label'].unique()))

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {v: k for k, v in label2id.items()}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(roberta_id)
config.update({"id2label": id2label})

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

## XLM Roberta

### Tokenization

In [ ]:
# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(xlm_roberta_id)

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
# https://huggingface.co/learn/nlp-course/chapter5/3?fw=pt#the-map-methods-superpowers
def tokenize(batch):
    return tokenizer(batch["combined_text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset = train_dataset.remove_columns(["combined_text"])
val_dataset = val_dataset.remove_columns(["combined_text"])
test_dataset = test_dataset.remove_columns(["combined_text"])

train_dataset = train_dataset.rename_column("emotion", "labels")
val_dataset = val_dataset.rename_column("emotion", "labels")
test_dataset = test_dataset.rename_column("emotion", "labels")


# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMRobertaTokenizerFast'. 
The class this function is called from is 'RobertaTokenizerFast'.


Map:   0%|          | 0/1149 [00:00<?, ? examples/s]

In [ ]:
train_dataset.select(range(50))

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 50
})

### Defining XLM-Roberta Model

In [ ]:
# Model
model = RobertaForSequenceClassification.from_pretrained(xlm_roberta_id, config=config)
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# TrainingArguments
training_args = TrainingArguments(
    output_dir="emotion_detection_xlm_roberta_base",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to=['tensorboard'],
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

In [ ]:
raw_inputs = [
    "سلام مونا روزگار چطور میگذرد ؟ خوبه ، از تو چی ؟ خوبه ، درس چطوره ؟ زیاد خوب نیست ، ریاضی مشکل دارم چه بد ، خب از یکی کمک بگیر ؛ معلمی دوستی میخواهم از معلم کمک بگیرم اما نگرانم نگران نباش وقت هست ممنونم امیدوارم اینگونه باشد خواهش میکنم فعلا فعلا",
    "وقت داری حرف بزنیم ؟ البته ؟ چی شده ؟ امروز نتونستم برم سر کار آیا اطلاع داده ای ؟ نه ، به دلیل مشکل پیش اومده نتونستم ارامش خودتو حفظ کن یعنی چکار کنم ؟ فردا به رییست توضیح بده و نگران نباش حتما مراقب خودت باش",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 5])


In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.2008, 0.1493, 0.2289, 0.2147, 0.2063],
        [0.2024, 0.1479, 0.2310, 0.2115, 0.2073]], grad_fn=<SoftmaxBackward0>)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer("Using a Transformer network is simple")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Evaluate the model
trainer.evaluate()

### Prediction


In [ ]:
# TEST MODEL

from transformers import pipeline

classifier = pipeline('text-classification',repository_id)

text = "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his innocence and vowing: quot;After the crucifixion comes the resurrection. quot; .."
result = classifier(text)

predicted_label = result[0]["label"]
print(f"Predicted label: {predicted_label}")

## Roberta

### Tokenization

In [9]:
# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(roberta_id)

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
# https://huggingface.co/learn/nlp-course/chapter5/3?fw=pt#the-map-methods-superpowers
def tokenize(batch):
    return tokenizer(batch["label"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")


# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/5512 [00:00<?, ? examples/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

### Defining Roberta model

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
train_dataset.features

{'labels': ClassLabel(names=['عصبانی', 'غمگین و مضطرب', ' عاشقانه و خوشحال', 'هیجانی و متعجب', 'معمولی'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
# Model
model = RobertaForSequenceClassification.from_pretrained(roberta_id, num_labels=5)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="emotion_detection_roberta_base",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to=['tensorboard'],
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
500,1.125800,1.071799,0.658834
1000,1.122900,1.083404,0.658834
1500,1.109000,1.075006,0.658834


TrainOutput(global_step=1941, training_loss=1.1169924925185553, metrics={'train_runtime': 1417.8153, 'train_samples_per_second': 21.881, 'train_steps_per_second': 1.369, 'total_flos': 4081357066830336.0, 'train_loss': 1.1169924925185553, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.0717992782592773,
 'eval_accuracy': 0.6588337684943429,
 'eval_runtime': 15.4878,
 'eval_samples_per_second': 74.187,
 'eval_steps_per_second': 4.649,
 'epoch': 3.0}

## MT5

### Tokenization

In [ ]:
from transformers import MT5TokenizerFast

train_dataset_tmp = train_dataset

# Preprocessing
tokenizer = MT5TokenizerFast.from_pretrained(mt5_id)

# This function tokenizes the input text using the RoBERTa tokenizer.
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
# https://huggingface.co/learn/nlp-course/chapter5/3?fw=pt#the-map-methods-superpowers
def tokenize(batch):
    return tokenizer(batch["combined_text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

train_dataset = train_dataset.remove_columns(["combined_text"])
val_dataset = val_dataset.remove_columns(["combined_text"])
test_dataset = test_dataset.remove_columns(["combined_text"])

train_dataset = train_dataset.rename_column("emotion", "labels")
val_dataset = val_dataset.rename_column("emotion", "labels")
test_dataset = test_dataset.rename_column("emotion", "labels")


# Set dataset format
train_dataset.set_format("torch", columns=["input_ids", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "labels"])

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Defining XLM-Roberta Model

In [ ]:
from transformers import MT5ForConditionalGeneration
# Model
model = MT5ForConditionalGeneration.from_pretrained(mt5_id, config=config)
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# TrainingArguments
training_args = TrainingArguments(
    output_dir="emotion_detection_mt5",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to=['tensorboard'],
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
raw_inputs = [
    "سلام مونا روزگار چطور میگذرد ؟ خوبه ، از تو چی ؟ خوبه ، درس چطوره ؟ زیاد خوب نیست ، ریاضی مشکل دارم چه بد ، خب از یکی کمک بگیر ؛ معلمی دوستی میخواهم از معلم کمک بگیرم اما نگرانم نگران نباش وقت هست ممنونم امیدوارم اینگونه باشد خواهش میکنم فعلا فعلا",
    "وقت داری حرف بزنیم ؟ البته ؟ چی شده ؟ امروز نتونستم برم سر کار آیا اطلاع داده ای ؟ نه ، به دلیل مشکل پیش اومده نتونستم ارامش خودتو حفظ کن یعنی چکار کنم ؟ فردا به رییست توضیح بده و نگران نباش حتما مراقب خودت باش",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)

torch.Size([2, 5])


In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.2008, 0.1493, 0.2289, 0.2147, 0.2063],
        [0.2024, 0.1479, 0.2310, 0.2115, 0.2073]], grad_fn=<SoftmaxBackward0>)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer("Using a Transformer network is simple")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Evaluate the model
trainer.evaluate()

### Prediction


In [ ]:
# TEST MODEL

from transformers import pipeline

classifier = pipeline('text-classification',repository_id)

text = "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his innocence and vowing: quot;After the crucifixion comes the resurrection. quot; .."
result = classifier(text)

predicted_label = result[0]["label"]
print(f"Predicted label: {predicted_label}")